In [1]:
import ccxt
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import matplotlib.pyplot as plt

import json
import time
import os
import sys

In [2]:
src_path = '../src/'
sys.path.append(os.path.abspath(src_path))
from func_get import *
from func_signal import *
from func_backtest import *

In [3]:
with open('../../../_keys/ftx/ftx_read_keys.json') as keys_file:
    keys_dict = json.load(keys_file)
    
exchange = ccxt.ftx({'apiKey': keys_dict['apiKey'],
                     'secret': keys_dict['secret'],
                     'enableRateLimit': True})

### Config

In [4]:
symbol = 'ETH-PERP'
timeframe = '15m'

### Get data

In [5]:
ohlcv_df = get_ohlcv_df(exchange, symbol, timeframe, since=None, limit=1000)
ohlcv_df = ohlcv_df.loc[:len(ohlcv_df) - 2, :]

In [6]:
ohlcv_df

,time,open,high,low,close,volume
0,2022-02-21 10:15:00,2730.9,2740.7,2723.1,2723.6,1.272632e+07
1,2022-02-21 10:30:00,2723.6,2725.9,2708.2,2714.0,2.166554e+07
2,2022-02-21 10:45:00,2714.0,2715.8,2704.6,2712.1,9.992556e+06
3,2022-02-21 11:00:00,2712.1,2727.4,2711.3,2723.5,1.640531e+07
4,2022-02-21 11:15:00,2723.5,2725.5,2717.8,2718.4,5.623949e+06
...,...,...,...,...,...,...
994,2022-03-03 18:45:00,2922.5,2922.5,2911.6,2913.6,1.107504e+07
995,2022-03-03 19:00:00,2913.6,2921.1,2904.9,2912.8,9.312756e+06
996,2022-03-03 19:15:00,2912.8,2932.3,2912.8,2929.6,1.744584e+07
997,2022-03-03 19:30:00,2929.6,2931.6,2914.5,2914.5,1.095766e+07


### Signal

### Donchian

In [ ]:
def add_donchian(ohlcv_df):
    def get_donchian_side(ohlcv_df):
        if ohlcv_df['close'] > ohlcv_df['max_high']:
            signal_side = 'buy'
        elif ohlcv_df['close'] < ohlcv_df['min_low']:
            signal_side = 'sell'
        else:
            signal_side = np.nan

        return signal_side
    
    
    signal_dict = {
        'windows': 30,
        'revert': False
    }
    
    windows = signal_dict['windows']
    
    temp_df = ohlcv_df.copy()

    max_high_list = temp_df['high'].rolling(window=signal_dict['windows']).max()
    min_low_list = temp_df['low'].rolling(window=signal_dict['windows']).min()
    
    temp_df['max_high'] = max_high_list
    temp_df['min_low'] = min_low_list
    
    temp_df['max_high'] = temp_df['max_high'].shift(periods=1)
    temp_df['min_low'] = temp_df['min_low'].shift(periods=1)
    temp_df['donchian'] = temp_df.apply(get_donchian_side, axis=1)
    
    donchian_list = temp_df['donchian'].fillna(method='ffill')
    ohlcv_df['donchian'] = donchian_list
    
    return ohlcv_df

In [ ]:
test_df = add_donchian(ohlcv_df)

In [ ]:
test_df

### Hull

In [7]:
def add_hull(ohlcv_df):
    def cal_wma(series):
        weight_list = list(range(1, len(series) + 1))
        weighted_average_list = [i * j for i, j in zip(series, weight_list)]

        wma = sum(weighted_average_list) / sum(weight_list)

        return wma
    
    
    def add_wma(df, input_col, windows):
        wma_list = [None] * (windows - 1)

        for i in range(len(df) - (windows - 1)):
            close_series = df.loc[i:i + (windows - 1), input_col]
            wma = cal_wma(close_series)
            wma_list.append(wma)

        return wma_list
    
    
    def get_hull_side(ohlcv_df):
        if ohlcv_df['hull'] > ohlcv_df['hull_prev']:
            signal_side = 'buy'
        else:
            signal_side = 'sell'

        return signal_side
    
    
    signal_dict = {
        'windows': 180,
        'revert': False
    }

    windows = signal_dict['windows']
    
    temp_df = ohlcv_df.copy()

    temp_df['hwma'] = add_wma(temp_df, 'close', int(round(windows / 2)))
    temp_df['wma'] = add_wma(temp_df, 'close', windows)
    temp_df['twma'] = (2 * temp_df['hwma']) - temp_df['wma']
    temp_df['hull'] = add_wma(temp_df, 'twma', int(round(windows ** (1 / 2))))
    temp_df['hull_prev'] = temp_df['hull'].shift(periods=2)
    temp_df['hull_side'] = temp_df.apply(get_hull_side, axis=1)
    
    ohlcv_df['hull_side'] = temp_df['hull_side']
    
    return ohlcv_df

In [8]:
test_df = add_hull(ohlcv_df)

In [12]:
test_df.to_csv("hull.csv")